In [324]:
:dep graph-ds = { path = ".", default-features = false }
:dep anyhow = "1.0"
// :dep h3o = "0.3.0"
// :dep plotters = "0.3.4"

In [325]:
use graph_ds::{Graph, Node, Edge};
use graph_ds::hexagon_graph::{cell_graph_from_mpk, cell::Cell};
// use h3o::Resolution;
use std::time::Instant;
use std::sync::{Arc, RwLock, RwLockWriteGuard};


use std::collections::hash_map::DefaultHasher;
use std::hash::{Hash, Hasher};


In [329]:
let mut graph : Graph<Cell> = cell_graph_from_mpk("../resources/de_mirage_hexagons.mpk")?;
println!("Graph has {} nodes", graph.nr_nodes());

Graph has 7935 nodes


In [330]:
graph.node_hash()

14821309618286519929

In [ ]:
fn heuristic(start_cell: &Cell, end_cell: &Cell) -> f64 {
    let dx = (start_cell.a - end_cell.a).abs();
    let dy = (start_cell.b - end_cell.b).abs();
    let dz = (start_cell.a + start_cell.b - end_cell.a - end_cell.b).abs();
    let dlayer = (start_cell.layer - end_cell.layer).abs();
    ((dx + dy + dz) as f64 / 2.0 + dlayer as f64) * (start_cell.radius * 2) as f64
}

In [ ]:
let start = graph.get_random_node().unwrap();
let end = graph.get_random_node().unwrap();

println!("Start: {:?}", start);
println!("End: {:?}", end);

Start: Cell { a: -9, b: -11, radius: 24, layer: 0 }
End: Cell { a: -39, b: -18, radius: 24, layer: -1 }


In [ ]:
println!("BFS start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let (bfs_path, bfs_distance) = graph.bfs(start, Some(end), &None)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", bfs_distance[0].unwrap());
println!("path length: {:?}", bfs_path.as_ref().unwrap().len());

println!("---");

// println!("matrix bfs");
// let now = Instant::now();
// graph.matrix_bfs_distance(vec![start; 10], Some(vec![end; 10]), true);
// println!("time: {:?} µs", now.elapsed().as_micros());
// let distances = distances.into_iter().flatten().collect::<Vec<_>>();
// println!("distances: {:?}", distances);

println!("---");

println!("AStar start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let (astar_path, astar_distance) = graph.astar(start, end, heuristic)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", astar_distance);
println!("path length: {:?}", astar_path.len());

BFS start: Cell { a: -9, b: -11, radius: 24, layer: 0 }, end: Cell { a: -39, b: -18, radius: 24, layer: -1 }
[backtrace] found start node
time: 670 µs
distance: 2642.506072998047
path length: 56
---
---
AStar start: Cell { a: -9, b: -11, radius: 24, layer: 0 }, end: Cell { a: -39, b: -18, radius: 24, layer: -1 }
[backtrace] found start node
time: 920 µs
distance: 2611.0711669921875
path length: 56


In [ ]:
println!("{:?}", bfs_path.unwrap());

[Cell { a: -9, b: -11, radius: 24, layer: 0 }, Cell { a: -10, b: -11, radius: 24, layer: 0 }, Cell { a: -10, b: -11, radius: 24, layer: -1 }, Cell { a: -11, b: -11, radius: 24, layer: -1 }, Cell { a: -12, b: -11, radius: 24, layer: -1 }, Cell { a: -13, b: -10, radius: 24, layer: -1 }, Cell { a: -14, b: -10, radius: 24, layer: -1 }, Cell { a: -15, b: -10, radius: 24, layer: -1 }, Cell { a: -16, b: -9, radius: 24, layer: -1 }, Cell { a: -17, b: -9, radius: 24, layer: -1 }, Cell { a: -18, b: -9, radius: 24, layer: -1 }, Cell { a: -19, b: -9, radius: 24, layer: -1 }, Cell { a: -20, b: -9, radius: 24, layer: -1 }, Cell { a: -21, b: -8, radius: 24, layer: -1 }, Cell { a: -22, b: -8, radius: 24, layer: -1 }, Cell { a: -22, b: -8, radius: 24, layer: 0 }, Cell { a: -23, b: -7, radius: 24, layer: 0 }, Cell { a: -24, b: -7, radius: 24, layer: 0 }, Cell { a: -25, b: -6, radius: 24, layer: 0 }, Cell { a: -26, b: -5, radius: 24, layer: 0 }, Cell { a: -27, b: -4, radius: 24, layer: 0 }, Cell { a: -28

In [ ]:
println!("{:?}", astar_path);

[Cell { a: -9, b: -11, radius: 24, layer: 0 }, Cell { a: -10, b: -11, radius: 24, layer: 0 }, Cell { a: -10, b: -11, radius: 24, layer: -1 }, Cell { a: -11, b: -10, radius: 24, layer: -1 }, Cell { a: -12, b: -9, radius: 24, layer: -1 }, Cell { a: -13, b: -8, radius: 24, layer: -1 }, Cell { a: -14, b: -7, radius: 24, layer: -1 }, Cell { a: -15, b: -6, radius: 24, layer: -1 }, Cell { a: -16, b: -5, radius: 24, layer: -1 }, Cell { a: -17, b: -5, radius: 24, layer: -1 }, Cell { a: -18, b: -4, radius: 24, layer: -1 }, Cell { a: -19, b: -4, radius: 24, layer: -1 }, Cell { a: -20, b: -3, radius: 24, layer: -1 }, Cell { a: -20, b: -3, radius: 24, layer: 0 }, Cell { a: -21, b: -3, radius: 24, layer: 0 }, Cell { a: -22, b: -2, radius: 24, layer: 0 }, Cell { a: -23, b: -2, radius: 24, layer: 0 }, Cell { a: -24, b: -1, radius: 24, layer: 0 }, Cell { a: -25, b: -1, radius: 24, layer: 0 }, Cell { a: -26, b: -1, radius: 24, layer: 0 }, Cell { a: -27, b: -1, radius: 24, layer: 0 }, Cell { a: -28, b: -